In [1]:
#Competition url:
    #https://www.kaggle.com/c/house-prices-advanced-regression-techniques

In [2]:
#Load packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from sklearn import linear_model
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
from numpy import sort
from collections import OrderedDict

C:\Users\Magu\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
#Import data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Training Data Cleaning

In [4]:
#Explore Data - Missing Data (1460 total training entries)

#train['LotFrontage'].unique()
#train['LotFrontage'].describe()
#train.info()
#train.describe()

#LotFrontage = 1201 float (Linear feet of street connected to property)

#Alley = 91 object [nan 'Grvl' 'Pave']

#MasVnrType = 1452 object ['BrkFace' 'None' 'Stone' 'BrkCmn' nan]
#MasVnrArea = 1452 float

#Electrical = 1459 object ['SBrkr' 'FuseF' 'FuseA' 'FuseP' 'Mix' nan]

#FireplaceQu = 770 object [nan 'TA' 'Gd' 'Fa' 'Ex' 'Po']

#GarageType = 1379 object ['Attchd' 'Detchd' 'BuiltIn' 'CarPort' nan 'Basment' '2Types']
#GarageYrBlt = 1379 float
#GarageFinish = 1379 object ['RFn' 'Unf' 'Fin' nan]
#GarageQual = 1379 object ['TA' 'Fa' 'Gd' nan 'Ex' 'Po']
#GarageCond = 1379 object ['TA' 'Fa' nan 'Gd' 'Po' 'Ex']

#PoolQC = 7 object [nan 'Ex' 'Fa' 'Gd']

#Fence = 281 object [nan 'MnPrv' 'GdWo' 'GdPrv' 'MnWw']

#MiscFeature = 54 object [nan 'Shed' 'Gar2' 'Othr' 'TenC']

#BsmtQual = 1423 object ['Gd' 'TA' 'Ex' nan 'Fa']
#BsmtCond = 1423 object ['TA' 'Gd' nan 'Fa' 'Po']
#BsmtExposure = 1422 object ['No' 'Gd' 'Mn' 'Av' nan]
#BsmtFinType1 = 1423 object ['GLQ' 'ALQ' 'Unf' 'Rec' 'BLQ' nan 'LwQ']
#BsmtFinType2 = 1422 object ['Unf' 'BLQ' nan 'ALQ' 'Rec' 'LwQ' 'GLQ']

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

In [6]:
#LotFrontage = 1201 float (Linear feet of street connected to property)
#Regression model to predict missing LotFrontage 

#Separate data into train and test sets
train_lot = train[np.isnan(train['LotFrontage']) == False]
test_lot = train[np.isnan(train['LotFrontage']) == True]
    
#Convert categorical using get_dummies (train data) 
dum_list = ['LotConfig','LandSlope','LandContour']
prefix_list = ['LotConfig','LandSlope','LandContour']
merged_data_dums = pd.DataFrame(train_lot['LotArea'])
for col in range(len(dum_list)):
    dums = pd.get_dummies(train_lot[dum_list[col]].astype('category'), prefix = prefix_list[col], drop_first=True)
    merged_data_dums = merged_data_dums.join(dums)
    
#Fit Lasso regression 
x = sm.add_constant(merged_data_dums)
y = train_lot['LotFrontage']
lassoreg = linear_model.Lasso()
lassoreg.fit(x,y)

#Convert categorical using get_dummies (test data) 
dum_list = ['LotConfig','LandSlope','LandContour']
prefix_list = ['LotConfig','LandSlope','LandContour']
merged_data_dums1 = pd.DataFrame(test_lot['LotArea'])
for col in range(len(dum_list)):
    dums = pd.get_dummies(test_lot[dum_list[col]].astype('category'), prefix = prefix_list[col], drop_first=True)
    merged_data_dums1 = merged_data_dums1.join(dums)

#Prdeict LotFrontage with Lasso regression 
X = sm.add_constant(merged_data_dums)
lot_pred_lasso = lassoreg.predict(X)

#Replace NaN LotFrontage with predicted values from Lasso regression
d = 0
for i in range(0,len(train)):
    if np.isnan(train['LotFrontage'].get_value(i)) == True:
        train.set_value(i,['LotFrontage'],lot_pred_lasso[d])
        d = d + 1

In [7]:
#Alley = 91 object [nan 'Grvl' 'Pave'] (Type of alley access to property)

#Replace nan with 'none'
alley_map = {np.NaN:'None'}
train['Alley'] = train['Alley'].apply(lambda r:alley_map.get(r) if r in alley_map else r)

In [8]:
#MasVnrType = 1452 object ['BrkFace' 'None' 'Stone' 'BrkCmn' nan]
#MasVnrArea = 1452 float

#Replace MasVnrType nan values with None
masvnrtype_map = {np.NaN:'None'}
train['MasVnrType'] = train['MasVnrType'].apply(lambda r:masvnrtype_map.get(r) if r in masvnrtype_map else r)

#Replace MasVnrArea nan values with 0
for i in range(0,len(train)):
    if np.isnan(train['MasVnrArea'].get_value(i)) == True:
        train.set_value(i,['MasVnrArea'],0)

In [9]:
#Electrical = 1459 object ['SBrkr' 'FuseF' 'FuseA' 'FuseP' 'Mix' nan]
#Replace missing value with mode
mode_elec = train['Electrical'].mode() #SBrkr
elec_map = {np.NaN:mode_elec}
train['Electrical'] = train['Electrical'].apply(lambda r:elec_map.get(r) if r in elec_map else r)

In [10]:
#FireplaceQu = 770 object [nan 'TA' 'Gd' 'Fa' 'Ex' 'Po']
#Replace missing value with None
fire_map = {np.NaN:'None'}
train['FireplaceQu'] = train['FireplaceQu'].apply(lambda r:fire_map.get(r) if r in fire_map else r)

In [11]:
#GarageType = 1379 object ['Attchd' 'Detchd' 'BuiltIn' 'CarPort' nan 'Basment' '2Types']
#GarageYrBlt = 1379 float
#GarageFinish = 1379 object ['RFn' 'Unf' 'Fin' nan]
#GarageQual = 1379 object ['TA' 'Fa' 'Gd' nan 'Ex' 'Po']
#GarageCond = 1379 object ['TA' 'Fa' nan 'Gd' 'Po' 'Ex']

#Check if GarageQual and GarageCond are the same
match = train['GarageQual'] == train['GarageCond'] # Not all True so not the same

#Find indecies of nan for each feature 
GarageType_index = train[train['GarageType'].isnull()].index.tolist()
GarageYrBlt_index = train[train['GarageYrBlt'].isnull()].index.tolist()
GarageFinish_index = train[train['GarageFinish'].isnull()].index.tolist()
GarageQual_index = train[train['GarageQual'].isnull()].index.tolist()
GarageCond_index = train[train['GarageCond'].isnull()].index.tolist()

#Check that nan for each feature is in the same row 
GarageType_index==GarageYrBlt_index==GarageFinish_index==GarageQual_index==GarageCond_index
    #True

#Replace GarageType, GarageFinish, GarageQual,GarageCond nan with None
garage_map = {np.NaN:'None'}
train['GarageType'] = train['GarageType'].apply(lambda r:garage_map.get(r) if r in garage_map else r)
train['GarageFinish'] = train['GarageFinish'].apply(lambda r:garage_map.get(r) if r in garage_map else r)
train['GarageQual'] = train['GarageQual'].apply(lambda r:garage_map.get(r) if r in garage_map else r)
train['GarageCond'] = train['GarageCond'].apply(lambda r:garage_map.get(r) if r in garage_map else r)

#Replace GarageYrBlt nan values with 0
for i in range(0,len(train)):
    if np.isnan(train['GarageYrBlt'].get_value(i)) == True:
        train.set_value(i,['GarageYrBlt'],0)

In [12]:
#PoolQC = 7 object [nan 'Ex' 'Fa' 'Gd']
#Replace missing value with None
pool_map = {np.NaN:'None'}
train['PoolQC'] = train['PoolQC'].apply(lambda r:pool_map.get(r) if r in pool_map else r)

In [13]:
#Fence = 281 object [nan 'MnPrv' 'GdWo' 'GdPrv' 'MnWw']
#Replace missing value with None
fence_map = {np.NaN:'None'}
train['Fence'] = train['Fence'].apply(lambda r:fence_map.get(r) if r in fence_map else r)

In [14]:
#MiscFeature = 54 object [nan 'Shed' 'Gar2' 'Othr' 'TenC']
#Replace missing value with None
misc_map = {np.NaN:'None'}
train['MiscFeature'] = train['MiscFeature'].apply(lambda r:misc_map.get(r) if r in misc_map else r)

In [15]:
#BsmtQual = 1423 object ['Gd' 'TA' 'Ex' nan 'Fa']
#BsmtCond = 1423 object ['TA' 'Gd' nan 'Fa' 'Po']
#BsmtExposure = 1422 object ['No' 'Gd' 'Mn' 'Av' nan]
#BsmtFinType1 = 1423 object ['GLQ' 'ALQ' 'Unf' 'Rec' 'BLQ' nan 'LwQ']
#BsmtFinType2 = 1422 object ['Unf' 'BLQ' nan 'ALQ' 'Rec' 'LwQ' 'GLQ']

#Find indexs of nan for each feature 
BsmtQual_index = train[train['BsmtQual'].isnull()].index.tolist()
BsmtCond_index = train[train['BsmtCond'].isnull()].index.tolist()
BsmtExposure_index = train[train['BsmtExposure'].isnull()].index.tolist()
BsmtFinType1_index = train[train['BsmtFinType1'].isnull()].index.tolist()
BsmtFinType2_index = train[train['BsmtFinType2'].isnull()].index.tolist()
#BsmtQual_index==BsmtCond_index==BsmtFinType1_index  
    #True

#Replace missing BsmtExposure with mode
d = 0
i = 0
while i < len(BsmtExposure_index) and d == 0:
    if (BsmtExposure_index[i]==BsmtQual_index[i])==False:
        BsmtExposure_miss_index = BsmtExposure_index[i]
        BsmtExposure_mode = train['BsmtExposure'].mode() #No
        train.set_value(BsmtExposure_miss_index,['BsmtExposure'],BsmtExposure_mode)
        d = 1
    i = i + 1

#Replace missing BsmtFinType2 with mode
d = 0
i = 0
while i < len(BsmtFinType2_index) and d == 0:
    if (BsmtFinType2_index[i]==BsmtQual_index[i])==False:
        BsmtFinType2_miss_index = BsmtFinType2_index[i]
        BsmtFinType2_mode = train['BsmtFinType2'].mode() #Unf
        train.set_value(BsmtFinType2_miss_index,['BsmtFinType2'],BsmtFinType2_mode)
        d = 1
    i = i + 1

In [16]:
#Replace nan BsmtFinType1 with None
BsmtFinType1_map = {np.NaN:'None'}
train['BsmtFinType1'] = train['BsmtFinType1'].apply(lambda r:BsmtFinType1_map.get(r) if r in BsmtFinType1_map else r)

#Replace nan BsmtQual with None
BsmtQual_map = {np.NaN:'None'}
train['BsmtQual'] = train['BsmtQual'].apply(lambda r:BsmtQual_map.get(r) if r in BsmtQual_map else r)

#Replace nan BsmtCond with None
BsmtCond_map = {np.NaN:'None'}
train['BsmtCond'] = train['BsmtCond'].apply(lambda r:BsmtCond_map.get(r) if r in BsmtCond_map else r)

#Replace nan BsmtFinType2 with None
for i in range(0,len(train)):
    if pd.isnull(train['BsmtFinType2'].get_value(i)):
        train.set_value(i,['BsmtFinType2'],'None')
        
#Replace nan BsmtExposure with None
for i in range(0,len(train)):
    if pd.isnull(train['BsmtExposure'].get_value(i)):
        train.set_value(i,['BsmtExposure'],'None')

In [17]:
#Exploring Data for Outliers
train.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.073490,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.117123,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,22.202524,9981.264932,1.382997,1.112799,30.202904,20.645407,180.731373,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,60.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.546209,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,79.801175,11601.500000,7.000000,6.000000,2000.000000,2004.000000,164.250000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [18]:
#Replace top end outliers with median
x_var_list = ['LotFrontage','LotArea','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','BsmtFullBath',     
'BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd','Fireplaces','GarageYrBlt','GarageCars','GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal']
for g in range(0,len(x_var_list)):
    A = train[x_var_list[g]].std()
    C = train[x_var_list[g]].mean()
    D = train[x_var_list[g]].median()
    for r in range(0,len(train[x_var_list[g]])):
        if train[x_var_list[g]].get_value(r)-C >= 3*A:
            train.set_value(r,[x_var_list[g]],D)

# Test Data Cleaning

In [19]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
Id               1459 non-null int64
MSSubClass       1459 non-null int64
MSZoning         1455 non-null object
LotFrontage      1232 non-null float64
LotArea          1459 non-null int64
Street           1459 non-null object
Alley            107 non-null object
LotShape         1459 non-null object
LandContour      1459 non-null object
Utilities        1457 non-null object
LotConfig        1459 non-null object
LandSlope        1459 non-null object
Neighborhood     1459 non-null object
Condition1       1459 non-null object
Condition2       1459 non-null object
BldgType         1459 non-null object
HouseStyle       1459 non-null object
OverallQual      1459 non-null int64
OverallCond      1459 non-null int64
YearBuilt        1459 non-null int64
YearRemodAdd     1459 non-null int64
RoofStyle        1459 non-null object
RoofMatl         1459 non-null object
Exterior1st      1458 non-

In [20]:
#Replace crap in test set
dum_list = ['MSZoning','Street','Alley','LotShape','LandContour','Utilities','LotConfig','LandSlope','Neighborhood','Condition1',
            'Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','ExterQual',
            'ExterCond','Foundation','BsmtQual','BsmtCond','BsmtFinType1','BsmtFinType2','Heating','HeatingQC',
            'CentralAir','Electrical','KitchenQual','Functional','FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond',
            'PavedDrive','PoolQC','Fence','MiscFeature','SaleType','SaleCondition','MSSubClass','OverallQual','OverallCond','YearBuilt','YearRemodAdd','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd','Fireplaces','GarageYrBlt','GarageCars']

for c in range(len(dum_list)):
    test[dum_list[c]] = test[dum_list[c]].replace(',','')
    test[dum_list[c]] = test[dum_list[c]].replace('<','')

In [21]:
#Regression model to predict missing LotFrontage 

#Separate data into train and test sets
train_lot = test[np.isnan(test['LotFrontage']) == False]
test_lot = test[np.isnan(test['LotFrontage']) == True]
    
#Convert categorical using get_dummies (train data) 
dum_list = ['LotConfig','LandSlope','LandContour']
prefix_list = ['LotConfig','LandSlope','LandContour']
merged_data_dums = pd.DataFrame(train_lot['LotArea'])
for col in range(len(dum_list)):
    dums = pd.get_dummies(train_lot[dum_list[col]].astype('category'), prefix = prefix_list[col], drop_first=True)
    merged_data_dums = merged_data_dums.join(dums)
    
#Fit Lasso regression 
x = sm.add_constant(merged_data_dums)
y = train_lot['LotFrontage']
lassoreg = linear_model.Lasso()
lassoreg.fit(x,y)

#Convert categorical using get_dummies (test data) 
dum_list = ['LotConfig','LandSlope','LandContour']
prefix_list = ['LotConfig','LandSlope','LandContour']
merged_data_dums1 = pd.DataFrame(test_lot['LotArea'])
for col in range(len(dum_list)):
    dums = pd.get_dummies(test_lot[dum_list[col]].astype('category'), prefix = prefix_list[col], drop_first=True)
    merged_data_dums1 = merged_data_dums1.join(dums)

#Prdeict LotFrontage with Lasso regression 
X = sm.add_constant(merged_data_dums)
lot_pred_lasso = lassoreg.predict(X)

#Replace NaN LotFrontage with predicted values from Lasso regression
d = 0
for i in range(0,len(test)):
    if np.isnan(test['LotFrontage'].get_value(i)) == True:
        test.set_value(i,['LotFrontage'],lot_pred_lasso[d])
        d = d + 1

In [22]:
#Alley
alley_map = {np.NaN:'None'}
test['Alley'] = test['Alley'].apply(lambda r:alley_map.get(r) if r in alley_map else r)

In [23]:
#Utilities
mode_util = test['Utilities'].mode() 
util_map = {np.NaN:mode_util[0]}
test['Utilities'] = test['Utilities'].apply(lambda r:util_map.get(r) if r in util_map else r)

In [24]:
#Exterior1st
mode_ext1 = test['Exterior1st'].mode() 
ext1_map = {np.NaN:mode_ext1[0]}
test['Exterior1st'] = test['Exterior1st'].apply(lambda r:ext1_map.get(r) if r in ext1_map else r)

In [25]:
#Exterior2nd
mode_ext2 = test['Exterior2nd'].mode() 
ext2_map = {np.NaN:mode_ext2[0]}
test['Exterior2nd'] = test['Exterior2nd'].apply(lambda r:ext2_map.get(r) if r in ext2_map else r)

In [26]:
#MasVnrType and MasVnrArea
MasVnrType_index = test[test['MasVnrType'].isnull()].index.tolist()
MasVnrArea_index = test[test['MasVnrArea'].isnull()].index.tolist()

d = 0
i = 0
while i < len(MasVnrType_index) and d == 0:
    if (MasVnrType_index[i]==MasVnrArea_index[i])==False:
        MasVnrType_miss_index = MasVnrType_index[i]
        MasVnrType_mode = test['MasVnrType'].mode() #No
        test.set_value(MasVnrType_miss_index,['MasVnrType'],MasVnrType_mode)
        d = 1
    i = i + 1

for i in range(0,len(test)):
    if pd.isnull(test['MasVnrType'].get_value(i)):
        test.set_value(i,['MasVnrType'],'None')
        
for i in range(0,len(test)):
    if pd.isnull(test['MasVnrArea'].get_value(i)):
        test.set_value(i,['MasVnrArea'],0)

In [27]:
#Find missing TotalBsmtSF, BsmtFinSF1, BsmtFinSF2, BsmtUnfSF and replace with median
for i in range(0,len(test)):
    m = pd.isnull(test['BsmtFinType1'].get_value(i)) == True
    #BsmtFinType1 and BsmtFinType2 do not have any missing values. All null values from no basement.
        #BsmtFinType1_index = test[test['BsmtFinType1'].isnull()].index.tolist()
        #BsmtFinType2_index = test[test['BsmtFinType2'].isnull()].index.tolist()
        #BsmtFinType1_index == BsmtFinType2_index # True
    if pd.isnull(test['TotalBsmtSF'].get_value(i)) == True and m == True:
        test.set_value(i,['TotalBsmtSF'],test['TotalBsmtSF'].median())
    if pd.isnull(test['BsmtFinSF1'].get_value(i)) == True and m == True:
        test.set_value(i,['BsmtFinSF1'],test['BsmtFinSF1'].median())   
    if pd.isnull(test['BsmtFinSF2'].get_value(i)) == True and m == True:
        test.set_value(i,['BsmtFinSF2'],test['BsmtFinSF2'].median())   
    if pd.isnull(test['BsmtUnfSF'].get_value(i)) == True and m == True:
        test.set_value(i,['BsmtUnfSF'],test['BsmtUnfSF'].median())

In [28]:
#Basement Features list of null values index
BsmtQual_index = test[test['BsmtQual'].isnull()].index.tolist()
BsmtCond_index = test[test['BsmtCond'].isnull()].index.tolist()
BsmtExposure_index = test[test['BsmtExposure'].isnull()].index.tolist()
BsmtFinType1_index = test[test['BsmtFinType1'].isnull()].index.tolist()
BsmtFinSF1_index = test[test['BsmtFinSF1'].isnull()].index.tolist()
BsmtFinType2_index = test[test['BsmtFinType2'].isnull()].index.tolist()
BsmtFinSF2_index = test[test['BsmtFinSF2'].isnull()].index.tolist()
BsmtUnfSF_index = test[test['BsmtUnfSF'].isnull()].index.tolist()
TotalBsmtSF_index = test[test['TotalBsmtSF'].isnull()].index.tolist()
    #Index lists show that null values are due to mising data in addition to homes without basements 

#Find and replace null values
for i in range(0,len(test)):
    if  test['TotalBsmtSF'].get_value(i) > 0: #Replace missing data with median or mode
        if pd.isnull(test['BsmtQual'].get_value(i)) == True:
            test.set_value(i,['BsmtQual'],test['BsmtQual'].mode)
        if pd.isnull(test['BsmtCond'].get_value(i)) == True:
            test.set_value(i,['BsmtCond'],test['BsmtCond'].mode)
        if pd.isnull(test['BsmtExposure'].get_value(i)) == True:
            test.set_value(i,['BsmtExposure'],test['BsmtExposure'].mode)
        if pd.isnull(test['BsmtFinType1'].get_value(i)) == True:
            test.set_value(i,['BsmtFinType1'],test['BsmtFinType1'].mode)
        if pd.isnull(test['BsmtFinType2'].get_value(i)) == True:
            test.set_value(i,['BsmtFinType2'],test['BsmtFinType2'].mode)
        #TotalBsmtSF, BsmtFinSF1, BsmtFinSF2, BsmtUnfSF null values replaced with median in cell above
    elif test['TotalBsmtSF'].get_value(i) == 0: #Replace data for homes without a basement with None
        if pd.isnull(test['BsmtQual'].get_value(i)) == True:
            test.set_value(i,['BsmtQual'],'None')
        if pd.isnull(test['BsmtCond'].get_value(i)) == True:
            test.set_value(i,['BsmtCond'],'None')
        if pd.isnull(test['BsmtExposure'].get_value(i)) == True: 
            test.set_value(i,['BsmtExposure'],'None')
        if pd.isnull(test['BsmtFinType1'].get_value(i)) == True: 
            test.set_value(i,['BsmtFinType1'],'None')
        if pd.isnull(test['BsmtFinType2'].get_value(i)) == True: 
            test.set_value(i,['BsmtFinType2'],'None')

In [29]:
#BsmtFullBath and BsmtHalfBath
for i in range(0,len(test)):
    if pd.isnull(test['BsmtFullBath'].get_value(i)) == True and test['TotalBsmtSF'].get_value(i) > 0:
        test.set_value(i,['BsmtFullBath'],test['BsmtFullBath'].median())
    elif pd.isnull(test['BsmtFullBath'].get_value(i)) == True and test['TotalBsmtSF'].get_value(i) == 0:
        test.set_value(i,['BsmtFullBath'],0)
        
for i in range(0,len(test)):
    if pd.isnull(test['BsmtHalfBath'].get_value(i)) == True and test['TotalBsmtSF'].get_value(i) > 0:
        test.set_value(i,['BsmtHalfBath'],test['BsmtHalfBath'].median())
    elif pd.isnull(test['BsmtHalfBath'].get_value(i)) == True and test['TotalBsmtSF'].get_value(i) == 0:
        test.set_value(i,['BsmtHalfBath'],0)

In [30]:
#KitchenQual
for i in range(0,len(test)):
    if pd.isnull(test['KitchenQual'].get_value(i)):
        test.set_value(i,['KitchenQual'],test['KitchenQual'].mode())

In [31]:
#Functional
for i in range(0,len(test)):
    if pd.isnull(test['Functional'].get_value(i)):
        test.set_value(i,['Functional'],'None')

In [32]:
#FireplaceQu
for i in range(0,len(test)):
    if pd.isnull(test['FireplaceQu'].get_value(i)):
        test.set_value(i,['FireplaceQu'],'None')

In [33]:
#PoolQC
for i in range(0,len(test)):
    if pd.isnull(test['PoolQC'].get_value(i)):
        test.set_value(i,['PoolQC'],'None')

In [34]:
#Fence
for i in range(0,len(test)):
    if pd.isnull(test['Fence'].get_value(i)):
        test.set_value(i,['Fence'],'None')

In [35]:
#MiscFeature
for i in range(0,len(test)):
    if pd.isnull(test['MiscFeature'].get_value(i)):
        test.set_value(i,['MiscFeature'],'None')

In [36]:
#SaleType
for i in range(0,len(test)):
    if pd.isnull(test['SaleType'].get_value(i)):
        test.set_value(i,['SaleType'],test['SaleType'].mode())

In [37]:
#MSZoning
mode_zone = test['MSZoning'].mode() 
zone_map = {np.NaN:mode_zone}
test['MSZoning'] = test['MSZoning'].apply(lambda r:zone_map.get(r) if r in zone_map else r)

In [38]:
#GarageCars       1458 non-null float64
#GarageArea       1458 non-null float64

#Replace missing GarageCars and GarageArea with median
GarageCars_index = test[test['GarageCars'].isnull()].index.tolist()
GarageArea_index = test[test['GarageArea'].isnull()].index.tolist()
test.set_value(GarageCars_index,['GarageCars'],0)
test.set_value(GarageArea_index,['GarageArea'],0)

#GarageQual       1381 non-null object
#GarageCond       1381 non-null object
#GarageFinish     1381 non-null object
#GarageYrBlt      1381 non-null float64
#GarageType       1383 non-null object

GarageQual_index = test[test['GarageQual'].isnull()].index.tolist()
GarageCond_index = test[test['GarageCond'].isnull()].index.tolist()
GarageFinish_index = test[test['GarageFinish'].isnull()].index.tolist()
GarageYrBlt_index = test[test['GarageYrBlt'].isnull()].index.tolist()
GarageType_index = test[test['GarageType'].isnull()].index.tolist()

GarageQual_index == GarageCond_index == GarageFinish_index == GarageYrBlt_index
    #True

for i in range(0,len(test)):
    if pd.isnull(test['GarageType'].get_value(i)) == False and pd.isnull(test['GarageQual'].get_value(i)) == True:
        test.set_value(i,['GarageQual'],test['GarageQual'].mode)

for i in range(0,len(test)):
    if pd.isnull(test['GarageQual'].get_value(i)):
        test.set_value(i,['GarageQual'],'None')
        
for i in range(0,len(test)):
    if pd.isnull(test['GarageType'].get_value(i)) == False and pd.isnull(test['GarageCond'].get_value(i)) == True:
        test.set_value(i,['GarageCond'],test['GarageCond'].mode)

for i in range(0,len(test)):
    if pd.isnull(test['GarageCond'].get_value(i)):
        test.set_value(i,['GarageCond'],'None')
        
for i in range(0,len(test)):
    if pd.isnull(test['GarageType'].get_value(i)) == False and pd.isnull(test['GarageFinish'].get_value(i)) == True:
        test.set_value(i,['GarageFinish'],test['GarageFinish'].mode)

for i in range(0,len(test)):
    if pd.isnull(test['GarageFinish'].get_value(i)):
        test.set_value(i,['GarageFinish'],'None')
        
for i in range(0,len(test)):
    if pd.isnull(test['GarageType'].get_value(i)) == False and pd.isnull(test['GarageYrBlt'].get_value(i)) == True:
        test.set_value(i,['GarageYrBlt'],test['GarageYrBlt'].median())
        
for i in range(0,len(test)):
    if pd.isnull(test['GarageYrBlt'].get_value(i)):
        test.set_value(i,['GarageYrBlt'],0)

for i in range(0,len(test)):
    if pd.isnull(test['GarageType'].get_value(i)):
        test.set_value(i,['GarageType'],'None')

In [39]:
#Replace top end outliers with median
x_var_list = ['LotFrontage','LotArea','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','BsmtFullBath',     
'BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd','Fireplaces','GarageYrBlt','GarageCars','GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal']
for g in range(0,len(x_var_list)):
    A = test[x_var_list[g]].std()
    C = test[x_var_list[g]].mean()
    D = test[x_var_list[g]].median()
    for r in range(0,len(test[x_var_list[g]])):
        if test[x_var_list[g]].get_value(r)-C >= 3*A:
            test.set_value(r,[x_var_list[g]],D)

In [40]:
#train['Exterior2nd'].unique()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
Id               1459 non-null int64
MSSubClass       1459 non-null int64
MSZoning         1459 non-null object
LotFrontage      1459 non-null float64
LotArea          1459 non-null float64
Street           1459 non-null object
Alley            1459 non-null object
LotShape         1459 non-null object
LandContour      1459 non-null object
Utilities        1459 non-null object
LotConfig        1459 non-null object
LandSlope        1459 non-null object
Neighborhood     1459 non-null object
Condition1       1459 non-null object
Condition2       1459 non-null object
BldgType         1459 non-null object
HouseStyle       1459 non-null object
OverallQual      1459 non-null int64
OverallCond      1459 non-null int64
YearBuilt        1459 non-null int64
YearRemodAdd     1459 non-null int64
RoofStyle        1459 non-null object
RoofMatl         1459 non-null object
Exterior1st      1459 n

# Baseline - GBR vs XGBR

In [41]:
#Convert categorical using get_dummies (train data)

cols = ['GarageYrBlt']
for c in range(len(cols)):
    train[cols[c]] = pd.to_numeric(train[cols[c]],downcast='integer')  

dum_list = ['MSZoning','Street','Alley','LotShape','LandContour','LotConfig','LandSlope','Neighborhood','Condition1',
            'Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','ExterQual',
            'ExterCond','Foundation','BsmtQual','BsmtCond','BsmtFinType1','BsmtFinType2','Heating','HeatingQC',
            'CentralAir','Electrical','KitchenQual','Functional','FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond',
            'PavedDrive','PoolQC','Fence','MiscFeature','SaleType','SaleCondition','MSSubClass','OverallQual','OverallCond','YearBuilt','YearRemodAdd','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd','Fireplaces','GarageYrBlt','GarageCars']
        
merged_data_dums = pd.DataFrame(train[['LotFrontage','LotArea','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal','MoSold','YrSold']])
for col in range(len(dum_list)):
    dums = pd.get_dummies(train[dum_list[col]].astype('str'),prefix = dum_list[col],drop_first=True)
    merged_data_dums = merged_data_dums.join(dums)

In [42]:
#Train Test Split
x = merged_data_dums
y = train.SalePrice
x_train,x_test, y_train, y_test = train_test_split(x,y,train_size = 0.6,random_state=0)

C:\Users\Magu\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [43]:
#GradientBoostingRegressor Fit and Predict
clf = GradientBoostingRegressor()
clf = clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)
print(mean_squared_error(np.log(y_test), np.log(y_pred))**0.5)
print(r2_score(y_test, y_pred))

0.145778916299
0.820802637292


In [44]:
#Load xgboost
#https://www.lfd.uci.edu/~gohlke/pythonlibs/#xgboost
!pip install xgboost-0.7-cp36-cp36m-win_amd64.whl
import xgboost
from xgboost import plot_importance

In [45]:
#XGBoostRegressor Fit and Predict
xgb = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.09, gamma=0, subsample=0.9,colsample_bytree=1, max_depth=7)
xgb.fit(x_train,y_train)
y_pred = xgb.predict(x_test)
print(mean_squared_error(np.log(y_test), np.log(y_pred))**0.5)
print(r2_score(y_test, y_pred))

0.136832020265
0.842410431967


# Feature Importance

In [46]:
#Feature Importance

feat = xgb.feature_importances_
#array([ 0.12589832,  0.07026883,  0.02901251,  0.0449827 ,  0.0053234 ,
#        0.04099015,  0.04551504,  0.02794783,  0.0167687 ,  0.        ,
#        0.04791057,  0.04019164,  0.01969657,  0.02155975,  0.00638808,
#        0.        ,  0.00159702,  0.        ,  0.00079851,  0.028214  ,

#Reset training data based on feature importance
X = x.filter(x.columns[np.nonzero(feat)],axis=1)

#Train Test Split
x_train,x_test, y_train, y_test = train_test_split(X,y,train_size = 0.6,random_state=0)

#XGBoostRegressor Fit and Predict
xgb = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.09, gamma=0, subsample=0.9,colsample_bytree=1, max_depth=7)
#increase max depth
xgb.fit(x_train,y_train)
y_pred = xgb.predict(x_test)
print(mean_squared_error(np.log(y_test), np.log(y_pred))**0.5)
print(r2_score(y_test, y_pred))

C:\Users\Magu\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


0.136706464226
0.858405279891


# Predict Sale Price

In [47]:
#Convert categorical using get_dummies (test data)

cols = ['GarageCars','FullBath','LotFrontage','LotArea','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','MoSold','YrSold','GarageYrBlt']
for c in range(len(cols)):
    test[cols[c]] = pd.to_numeric(test[cols[c]],downcast='integer')  
    
merged_data_dums = pd.DataFrame(test[['LotFrontage','LotArea','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal','MoSold','YrSold']])

for col in range(len(dum_list)):
    dums = pd.get_dummies(test[dum_list[col]].astype('str'),prefix = dum_list[col],drop_first=True)
    merged_data_dums = merged_data_dums.join(dums)    

In [48]:
#XGBoostRegressor Fit and Predict
xgb = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.09, gamma=0, subsample=0.9,colsample_bytree=1, max_depth=7)
xgb.fit(x,train.SalePrice)
Xx = merged_data_dums
y_pred = xgb.predict(Xx)

ValueError: feature_names may not contain [, ] or <

In [ ]:
#Feature Engineering - TBD 
#x_sqrd = pd.DataFrame(train[['LotFrontage','LotArea','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea']]**2)
#x_sqrt = pd.DataFrame(train[['LotFrontage','LotArea','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea']]**(1/2))

In [ ]:
#Optimize with GridSearchCV


In [ ]:
#Export predicitons to csv
sub = pd.DataFrame(OrderedDict({'Id':test.Id,'SalePrice':y_pred}))
sub.to_csv('house_price_v6.csv', index=False)